In [1]:
# Dependencies and config file
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key
import openweathermapy as owm
from citipy import citipy
from random import randint
import numpy as np

In [13]:
# Randomly select 500 unique latitudes
latitude_list = []

for x in range(2500):
    latitude_list.append(np.random.uniform(low=-180.0, high=180, size=None))

# Convert list to series
latitude_list_series = pd.Series(latitude_list)

In [14]:
# Randomly select 500 unique longitudes
longitude_list = []

for x in range(2500):
    longitude_list.append(np.random.uniform(low=-90, high = 90))
    #print(longitude)

# Convert list to series
longitude_list_series = pd.Series(longitude_list)

In [4]:
# Concatentate the series into a df
lat_and_long = pd.concat([latitude_list_series, longitude_list_series], axis=1)
#lat_and_long.head(10)
#lat_and_long = lat_and_long.set_index(0)
lat_and_long.head(10)

,0,1
0,-10.170973,11.859185
1,104.827018,-26.752531
2,-153.491044,-17.254405
3,83.346982,73.607719
4,63.505567,-37.318198
5,-138.603074,-12.644378
6,171.261805,-8.914846
7,94.579902,29.765270
8,174.234104,26.793438
9,-76.623441,-79.800208


In [5]:
# Create new empty columns and add to existing dataframe 
lat_and_long["City"] = ("")
lat_and_long["Temperature"] =("")
lat_and_long["Cloudiness"] =("")
lat_and_long["Windspeed"] =("")
lat_and_long.head(10)
# Rename columns
lat_and_long = lat_and_long.rename(columns={0: "Latitude", 1:"Longitude"})
lat_and_long.head()

,Latitude,Longitude,City,Temperature,Cloudiness,Windspeed
0,-10.170973,11.859185,,,,
1,104.827018,-26.752531,,,,
2,-153.491044,-17.254405,,,,
3,83.346982,73.607719,,,,
4,63.505567,-37.318198,,,,


In [6]:
# Blank column for holiding the city name
cities = []

#row_counter = 0
for index, row in lat_and_long.iterrows():
    x = row["Latitude"]
    y = row["Longitude"]
    #city = citipy.nearest_city(x,y)
    lat_and_long.set_value(index, "City", citipy.nearest_city(x, y).city_name)
    #city_list = city.city_name
    #cities.append(city_list)
lat_and_long
#print(len(city_list))
#print(cities)
#len(cities)
#cities = pd.Series(cities)

,Latitude,Longitude,City,Temperature,Cloudiness,Windspeed
0,-10.170973,11.859185,luanda,,,
1,104.827018,-26.752531,illoqqortoormiut,,,
2,-153.491044,-17.254405,ushuaia,,,
3,83.346982,73.607719,dikson,,,
4,63.505567,-37.318198,tasiilaq,,,
5,-138.603074,-12.644378,ushuaia,,,
6,171.261805,-8.914846,barentsburg,,,
7,94.579902,29.765270,longyearbyen,,,
8,174.234104,26.793438,longyearbyen,,,
9,-76.623441,-79.800208,ushuaia,,,


In [7]:
lat_and_long = lat_and_long.drop_duplicates(subset = ["City"], keep = "first")
lat_and_long = lat_and_long.reset_index(drop=True)
lat_and_long

,Latitude,Longitude,City,Temperature,Cloudiness,Windspeed
0,-10.170973,11.859185,luanda,,,
1,104.827018,-26.752531,illoqqortoormiut,,,
2,-153.491044,-17.254405,ushuaia,,,
3,83.346982,73.607719,dikson,,,
4,63.505567,-37.318198,tasiilaq,,,
5,171.261805,-8.914846,barentsburg,,,
6,94.579902,29.765270,longyearbyen,,,
7,-106.756467,47.939904,port alfred,,,
8,-31.458165,25.412416,middelburg,,,
9,-11.223068,-36.508796,barra dos coqueiros,,,


In [17]:
# Create settings dictionary 
settings = {"units": "imperial", "APPID": api_key}
# Perform a weather check on each of the cities using a series of successive API calls
current_weather_luanda = owm.get_current("luanda", **settings)

# Include a print log of each city as it is being processed with the city number, city name, and requested URL.
print(f"Current weather object for luanda: {current_weather_luanda}.")


Current weather object for luanda: {'coord': {'lon': 13.24, 'lat': -8.83}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 87.8, 'pressure': 1009, 'humidity': 70, 'temp_min': 87.8, 'temp_max': 87.8}, 'visibility': 10000, 'wind': {'speed': 9.17, 'deg': 290}, 'clouds': {'all': 75}, 'dt': 1522065600, 'sys': {'type': 1, 'id': 6821, 'message': 0.0032, 'country': 'AO', 'sunrise': 1522041044, 'sunset': 1522084453}, 'id': 2240449, 'name': 'Luanda', 'cod': 200}.


In [9]:
# Save a CSV of all data retrieved

In [11]:
# Your objective is to build a series of scatter plots to showcase the following relationships:

# Temperature (F) vs. Latitude
# Humidity (%) vs. Latitude
# Cloudiness (%) vs. Latitude
# Wind Speed (mph) vs. Latitude

In [12]:
# Build a scatter plot for each data type

#plt.scatter(df["temperature"].astype(int),
            #df["latitude"],
            #edgecolor="black", linewidths=1, marker="o",
            #alpha=0.8, label="?")

# Label the graph and set axis limits
#plt.title("")
#plt.ylabel("")
#plt.xlabel("")
#plt.grid(True)
#plt.xlim([, ])
#plt.ylim([, ])

# Save the figure as png image (for each scatter plot) 
#plt.savefig("UnemploymentRate_BankCount.png")

# Show plot
#plt.show()